In [2]:
%load_ext sql
%sql postgresql://appdev@data/appdev

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: appdev@appdev'

# Materialised joins

## 1. `circuits` table report:

The view pg_indexes provides access to useful information about each index in the database.

In [87]:
%sql select * from pg_indexes where tablename = 'circuits';

3 rows affected.


schemaname,tablename,indexname,tablespace,indexdef
f1db,circuits,idx_17102_url,None,CREATE UNIQUE INDEX idx_17102_url ON circuits USING btree (url)
f1db,circuits,circuits_position_idx,None,"CREATE INDEX circuits_position_idx ON circuits USING gist (""position"")"
f1db,circuits,idx_17102_primary,None,CREATE UNIQUE INDEX idx_17102_primary ON circuits USING btree (circuitid)


* Type of indices and why they are of that type?

`idx_17102_url` index is of btree (url)

BTREE is the default index type for most storage engines.
The general idea of a B-Tree is that all the values are stored in order, and each leaf page is the same distance from the root. B Tree index speeds up the data access because storage engine don't have to scan the whole table instead it will start from root node.

`circuits_position_idx` index is of gist ("position")

GIST allows to generalise the search structures, so we can build indexes on type we choose. In this case GIST index is with <key, pointer> pairs using position instead of integers (like in B-trees).

`idx_17102_primary` index is of btree (circuitid)

In this case index is of BTREE on `circuitid` column.

* The amount of space each index takes up?

In [97]:
%sql select pg_size_pretty(pg_relation_size('idx_17102_url'));

1 rows affected.


pg_size_pretty
16 kB


In [92]:
%sql select pg_size_pretty(pg_relation_size('circuits_position_idx'));

1 rows affected.


pg_size_pretty
8192 bytes


In [93]:
%sql select pg_size_pretty(pg_relation_size('idx_17102_primary'));

1 rows affected.


pg_size_pretty
16 kB


## 2. Query of race winners

The query should return winner of all the races, but only if they are younger than 38 years:
* the date, 
* driver surname, 
* driver age, 
* track time in milliseconds, 
* race name,
* circuit name for all races.

In [232]:
%%sql
SELECT races.date, drivers.surname as "Driver's Surname", 
date_part('year', age('2018-03-19',dob)) as "Driver's Age", 
races.time as "Track Time(ms)", 
races.name as "Name", 
circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38
LIMIT 10;

10 rows affected.


date,Driver's Surname,Driver's Age,Track Time(ms),Name,Circuit Name
2008-03-16,Kovalainen,36.0,04:30:00,Australian Grand Prix,Albert Park Grand Prix Circuit
2008-04-06,Kovalainen,36.0,11:30:00,Bahrain Grand Prix,Bahrain International Circuit
2008-08-24,Massa,36.0,12:00:00,European Grand Prix,Valencia Street Circuit
2008-10-12,Massa,36.0,04:30:00,Japanese Grand Prix,Fuji Speedway
2008-10-19,Hamilton,33.0,07:00:00,Chinese Grand Prix,Shanghai International Circuit
2008-11-02,Massa,36.0,17:00:00,Brazilian Grand Prix,Autódromo José Carlos Pace
2007-04-08,Hamilton,33.0,07:00:00,Malaysian Grand Prix,Sepang International Circuit
2007-04-15,Massa,36.0,11:30:00,Bahrain Grand Prix,Bahrain International Circuit
2007-05-13,Massa,36.0,12:00:00,Spanish Grand Prix,Circuit de Barcelona-Catalunya
2007-05-27,Alonso,36.0,12:00:00,Monaco Grand Prix,Circuit de Monaco


## 3. Describe the query using EXPLAIN ANALYZE .

In [231]:
%%sql EXPLAIN ANALYZE 
SELECT races.date, drivers.surname as "Driver's Surname", 
date_part('year', age('2018-03-19',dob)) as "Driver's Age", 
races.time as "Track Time(ms)", 
races.name as "Name", 
circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38;

22 rows affected.


QUERY PLAN
Hash Join (cost=77.92..792.94 rows=86 width=66) (actual time=17.358..28.367 rows=154 loops=1)
Hash Cond: (races.circuitid = circuits.circuitid)
-> Hash Join (cost=74.28..787.57 rows=86 width=50) (actual time=16.210..25.429 rows=154 loops=1)
Hash Cond: (results.raceid = races.raceid)
-> Hash Join (cost=34.32..746.53 rows=86 width=19) (actual time=1.030..8.539 rows=154 loops=1)
Hash Cond: (results.driverid = drivers.driverid)
-> Seq Scan on results (cost=0.00..708.96 rows=257 width=16) (actual time=0.015..5.321 rows=257 loops=1)
Filter: (rank = 1)
Rows Removed by Filter: 23420
-> Hash (cost=30.82..30.82 rows=280 width=19) (actual time=0.984..0.984 rows=60 loops=1)


* How many calls are you making?

Here we can see that there are 10 arrows which means there are 10 calls made from the EXPLAIN ANALYZE call. This command displays the execution plan than shows how the table(s) referenced by the statement will be scanned.  
In this case multiple tables are referenced with join algorithms, that will be used to bring together the required rows from each input table.  
The most critical part of the display is the estimated statement execution cost, which is the planner's guess at how long it will take to run the statement.

* How long does it take to perform the query?

The last line displays the execution time coming from  ANALYZE command.  For each time running the query, the execution time varies.  
`Execution time: 29.298 ms`

## 4. Materialized view of your query.

Using EXPLAIN ANALYZE try to query the view. Write at least 5 lines of text explaining what's going on and why the query execution time changed.

In [233]:
%%sql CREATE VIEW race_winners AS
SELECT races.date, drivers.surname as "Driver's Surname", 
date_part('year', age('2018-03-19',dob)) as "Driver's Age", 
races.time as "Track Time(ms)", 
races.name as "Name", 
circuits.name as "Circuit Name" 
FROM results
JOIN drivers USING (driverid)
JOIN races USING (raceId)
JOIN circuits USING (circuitid)
WHERE results.rank = 1
AND date_part('year', age('2018-03-19',dob)) < 38;

Done.


[]

#### Views or `stored` query 
Makes it possible to persist some of your queries.  
Views are a logical virtual table created by “select query” but the result is not stored anywhere in the disk and every time we need to fire the query when we need data, so always we get updated or latest data from original tables.  
A view contains rows and columns, just like a real table. The fields in a view are fields from one or more real tables in the database.  
A view always shows up-to-date data! The database engine recreates the data, using the view's SQL statement, every time a user queries a view.  
Views allow a level of separation than an original table in terms of access rights but it always fetches updated data.  
`Execution time: 36.269 ms`

In [239]:
%sql EXPLAIN ANALYZE SELECT * FROM race_winners;

22 rows affected.


QUERY PLAN
Hash Join (cost=77.92..792.94 rows=86 width=66) (actual time=18.278..34.971 rows=154 loops=1)
Hash Cond: (races.circuitid = circuits.circuitid)
-> Hash Join (cost=74.28..787.57 rows=86 width=50) (actual time=17.084..31.278 rows=154 loops=1)
Hash Cond: (results.raceid = races.raceid)
-> Hash Join (cost=34.32..746.53 rows=86 width=19) (actual time=1.167..13.178 rows=154 loops=1)
Hash Cond: (results.driverid = drivers.driverid)
-> Seq Scan on results (cost=0.00..708.96 rows=257 width=16) (actual time=0.018..9.196 rows=257 loops=1)
Filter: (rank = 1)
Rows Removed by Filter: 23420
-> Hash (cost=30.82..30.82 rows=280 width=19) (actual time=1.110..1.110 rows=60 loops=1)


#### Materialised views 
Materialized views are also the logical view of our data-driven by the select query but the result of the query will get stored in the table or disk, also the definition of the query will also store in the database.  
Use the `CREATE MATERIALIZED VIEW` statement to create a materialized view. A materialized view is a database object that contains the results of a query, they will NOT update if your data is updated.  
When we see the performance of Materialized view it is better than normal View because the data of materialized view will be stored in table and table may be indexed so faster for joining.
`Execution time: 2.201 ms`

In [237]:
%sql CREATE MATERIALIZED VIEW race_winners_cache AS SELECT * FROM race_winners;

154 rows affected.


[]

In [238]:
%sql EXPLAIN ANALYZE SELECT * FROM race_winners_cache;

3 rows affected.


QUERY PLAN
Seq Scan on race_winners_cache (cost=0.00..10.50 rows=50 width=1568) (actual time=0.031..1.107 rows=154 loops=1)
Planning time: 9.654 ms
Execution time: 2.201 ms
